<a href="https://colab.research.google.com/github/antonellafontanetto/DataScience-Entregas/blob/main/Proyecto_YPF_4x4_AntoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto 4x4 YPF: Primer Pilar Vaca Muerta a corto plazo

El siguiente proyecto está focalizado en predecir el comportamiento de determinadas variables en el marco del proyecto 4x4 de YPF, el cual consta de cuadriplicar la empresa en los próximos 4 años a través de la expansión de la producción, eficiencia en los procesos y mayor foco en la generación de valor.

Para lo cual el proyecta va a constar del siguiente procedimiento:

1) Recolección de Datos

2) Exploración y Procesamiento de los datos (exploración de los datos, discreción de variables, normalización, limpieza, visualización previa)

3) Modelado (construir y testear modelos para predecir o clasificar información o encontrar patrones de datos)

4) Puesta en Producción (predecir nuevos datos, comunicar los resultados, integrar los resultados con aplicación)


**1) Recolección de datos**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
#Cargo los datasets que vamos a utilizar en el proyecto
produccion = pd.read_csv('/content/produccion.csv', usecols=['anio','empresa','tipopozo','tipoestado','prod_gas','prod_pet','profundidad','areayacimiento','cuenca','provincia','tipo_de_recurso','sub_tipo_recurso'])
pozosterminados = pd.read_csv('/content/pozosterminados.csv', usecols=['anio','empresa','areapermisoconcesion','areayacimiento','cuenca','provincia','ubicacion','tipodepozoterminado','concepto'])
produccion
pozosterminados

,anio,empresa,areapermisoconcesion,areayacimiento,cuenca,provincia,ubicacion,tipodepozoterminado,concepto
0,2009,ALIANZA PETROLERA ARGENTINA S.A.,CALMUCO - BARREALES COLORADOS,BARREALES COLORADOS,NEUQUINA,Mendoza,On Shore,Explotación,Productivos de Petróleo
1,2009,COMPAÑÍAS ASOCIADAS PETROLERAS S.A.,DIADEMA,DIADEMA,GOLFO SAN JORGE,Chubut,On Shore,Explotación,Productivos de Gas
2,2009,EZ HOLDINGS S.A.,CAÑADON PILAR,CAÑADON PILAR,GOLFO SAN JORGE,Chubut,On Shore,Exploración,Productivos de Petróleo
3,2009,YPF S.A.,CERRO PIEDRA - CERRO GUADAL NORTE,CERRO GUADAL NORTE,GOLFO SAN JORGE,Santa Cruz,On Shore,Servicio,Servicio
4,2009,ALIANZA PETROLERA ARGENTINA S.A.,CALMUCO - BARREALES COLORADOS,BARREALES COLORADOS,NEUQUINA,Mendoza,On Shore,Explotación,Productivos de Gas
...,...,...,...,...,...,...,...,...,...
1048305,2024,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,EL JEFE,GOLFO SAN JORGE,Chubut,On Shore,Explotación,Productivos de Petróleo
1048306,2024,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,PUESTO VERA,GOLFO SAN JORGE,Chubut,On Shore,Explotación,Improductivos
1048307,2024,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,PUESTO VERA,GOLFO SAN JORGE,Chubut,On Shore,Explotación,Productivos de Petróleo
1048308,2024,COLHUE HUAPI S.A.,ESTANCIA LA ESCONDIDA,ESTANCIA LA ESCONDIDA,GOLFO SAN JORGE,Chubut,On Shore,Explotación,Productivos de Petróleo


In [5]:
#Filtro por cada dataset solo los registros para YPF S.A.
prod_ypf = produccion[(produccion['empresa'] == 'YPF S.A.')&(produccion['cuenca'] == 'NEUQUINA')&(produccion['anio']>2022)]

prod_ypf


,anio,prod_pet,prod_gas,tipoestado,tipopozo,empresa,profundidad,areayacimiento,cuenca,provincia,tipo_de_recurso,sub_tipo_recurso
8572,2025,32.29,172.10,Extracción Efectiva,Gasífero,YPF S.A.,3844.0,ESTACION FERNANDEZ ORO,NEUQUINA,Rio Negro,NO CONVENCIONAL,TIGHT
8578,2025,0.00,0.00,Abandonado,Otro tipo,YPF S.A.,2827.0,EL OREJANO,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8584,2025,0.00,0.00,En Estudio,Otro tipo,YPF S.A.,2977.0,LA AMARGA CHICA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8590,2025,0.00,0.00,Abandonado,Otro tipo,YPF S.A.,4092.0,EL OREJANO,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
8596,2025,55.99,3.37,Extracción Efectiva,Petrolífero,YPF S.A.,4945.0,LOMA CAMPANA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
...,...,...,...,...,...,...,...,...,...,...,...,...
199438,2024,0.00,0.00,En Inyección Efectiva,Sumidero,YPF S.A.,2046.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199440,2024,1642.63,101.18,Extracción Efectiva,Petrolífero,YPF S.A.,5889.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199442,2024,2441.10,153.31,Extracción Efectiva,Petrolífero,YPF S.A.,0.0,LOMA COLORADA,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE
199446,2024,0.00,0.00,Parado Transitoriamente,Gasífero,YPF S.A.,4491.0,LAS TACANAS,NEUQUINA,Neuquén,NO CONVENCIONAL,SHALE


In [6]:
pozos_ypf = pozosterminados[(pozosterminados['empresa'] == 'YPF S.A.')&(pozosterminados['cuenca'] == 'NEUQUINA')&(produccion['anio']>2022)]
pozos_ypf

,anio,empresa,areapermisoconcesion,areayacimiento,cuenca,provincia,ubicacion,tipodepozoterminado,concepto
8734,2010,YPF S.A.,SEÑAL PICADA - PUNTA BARDA,SEÑAL PICADA,NEUQUINA,Rio Negro,On Shore,Explotación,Productivos de Gas
10384,2010,YPF S.A.,SEÑAL PICADA - PUNTA BARDA,SEÑAL PICADA,NEUQUINA,Rio Negro,On Shore,Explotación,Productivos de Petróleo
10408,2010,YPF S.A.,CAÑADON AMARILLO,CAÑADON AMARILLO,NEUQUINA,Mendoza,On Shore,Explotación,Improductivos
10420,2010,YPF S.A.,LOMA DE LA MINA,LOMA DE LA MINA,NEUQUINA,Mendoza,On Shore,Avanzada,Improductivos
10456,2010,YPF S.A.,EL MEDANITO,EL MEDANITO,NEUQUINA,Rio Negro,On Shore,Explotación,Improductivos
...,...,...,...,...,...,...,...,...,...
345377,2015,YPF S.A.,LA RIBERA BLOQUE II,LA RIBERA BLOQUE II,NEUQUINA,Neuquén,On Shore,Exploración,Productivos de Gas
345378,2015,YPF S.A.,BARRANCA DE LOS LOROS,SUBIDA BARDA CHICA,NEUQUINA,Rio Negro,On Shore,Exploración,Productivos de Gas
345379,2015,YPF S.A.,LA RIBERA BLOQUE I,LA RIBERA BLOQUE I,NEUQUINA,Neuquén,On Shore,Exploración,Productivos de Gas
345380,2015,YPF S.A.,LAS MANADAS,CERRO BAYITO,NEUQUINA,Neuquén,On Shore,Exploración,Productivos de Gas


In [ ]:
#Unir los datasets por features en común
petroleo = pd.merge(prod_ypf, pozos_ypf, on='empresa',how='inner')
petroleo